In [1]:
import numpy as np
import pandas as pd
from BasketballTree import Datapoint
from BasketballTree import KDTree
from BasketballTree import Node

In [2]:
df = pd.read_feather('../Datasets/DUKEMBB/duke_mbb_history_stats.feather')
df

,pid,gs,gspos,minutes,fg,fga,fg3,fg3a,ft,fta,...,tovers,blk,stl,tech,fgp,fg3p,ftp,year,month,day
0,500,False,None,0.0,0,0.0,0.0,0.0,4,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1906,3,2
1,502,False,None,0.0,2,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1906,3,2
2,503,False,None,0.0,1,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1906,3,2
3,504,False,None,0.0,0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1906,3,2
4,505,False,None,0.0,0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1906,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24133,2774,True,f,31.0,7,14.0,1.0,4.0,1,2,...,2.0,2.0,1.0,0.0,0.500000,0.250000,0.50,2017,1,23
24134,2757,True,g,33.0,5,13.0,5.0,11.0,4,4,...,4.0,0.0,1.0,1.0,0.384615,0.454545,1.00,2017,1,28
24135,2766,True,g,33.0,11,14.0,6.0,6.0,6,10,...,3.0,0.0,0.0,0.0,0.785714,1.000000,0.60,2017,1,28
24136,2773,False,None,24.0,3,7.0,1.0,3.0,2,2,...,0.0,0.0,0.0,0.0,0.428571,0.333333,1.00,2017,1,28


In [3]:
test = df[['pid', 'fg', 'day', 'month', 'year']].copy()
test.loc[:, 'timestamp'] = test['year'].astype(str) + test['month'].astype(str).str.zfill(2) + test['day'].astype(str).str.zfill(2)
test['timestamp'] = test['timestamp'].astype(int)
test['fg'] = test['fg'].astype(int)
test

,pid,fg,day,month,year,timestamp
0,500,0,2,3,1906,19060302
1,502,2,2,3,1906,19060302
2,503,1,2,3,1906,19060302
3,504,0,2,3,1906,19060302
4,505,0,2,3,1906,19060302
...,...,...,...,...,...,...
24133,2774,7,23,1,2017,20170123
24134,2757,5,28,1,2017,20170128
24135,2766,11,28,1,2017,20170128
24136,2773,3,28,1,2017,20170128


In [4]:
datapoints1 = list()
for row in test.itertuples():
    dp1 = Datapoint(coordinate=[row.timestamp, row.fg], pid=row.pid)
    datapoints1.append(dp1)

tree1 = KDTree()
tree1.build_tree(datapoints1)

In [5]:
result = test.query("timestamp > 19060301 and timestamp < 19726197")
result

,pid,fg,day,month,year,timestamp
0,500,0,2,3,1906,19060302
1,502,2,2,3,1906,19060302
2,503,1,2,3,1906,19060302
3,504,0,2,3,1906,19060302
4,505,0,2,3,1906,19060302
...,...,...,...,...,...,...
17972,137,5,30,12,1972,19721230
17973,184,3,30,12,1972,19721230
17974,257,6,30,12,1972,19721230
17975,281,5,30,12,1972,19721230


In [6]:
c_nodes = tree1.query_canonical( ([19060302, 0], [19361211, 2]) )

In [7]:
def print_nodes(nodes: list[Node]):
    if len(nodes) == 0:
        return
    for node in nodes:
        if node.datapoint is not None:
            print(node.datapoint.coordinate, node.datapoint.kwargs['pid'])
        else:
            print_nodes([node.left_child])
            print_nodes([node.right_child])
            

In [8]:
print_nodes(c_nodes)

[19060302, 0] 500
[19060302, 0] 504
[19060302, 0] 505
[19060302, 0] 506
[19060302, 1] 503
[19361211, 0] 421
[19361211, 1] 77
[19361211, 1] 178
[19361211, 0] 185
[19361211, 0] 343
[19060302, 2] 502
[19361211, 1] 263
[19361211, 1] 290
[19361212, 1] 263
[19361212, 1] 421
[19361211, 2] 340


In [9]:
k = 5
window = [19060302 , 19726197]
data = df[['pid', 'fg', 'day', 'month', 'year']].copy()
data.loc[:, 'timestamp'] = data['year'].astype(str) + data['month'].astype(str).str.zfill(2) + data['day'].astype(str).str.zfill(2)
data['timestamp'] = data['timestamp'].astype(int)
data['fg'] = data['fg'].astype(int)
data

,pid,fg,day,month,year,timestamp
0,500,0,2,3,1906,19060302
1,502,2,2,3,1906,19060302
2,503,1,2,3,1906,19060302
3,504,0,2,3,1906,19060302
4,505,0,2,3,1906,19060302
...,...,...,...,...,...,...
24133,2774,7,23,1,2017,20170123
24134,2757,5,28,1,2017,20170128
24135,2766,11,28,1,2017,20170128
24136,2773,3,28,1,2017,20170128


In [10]:
def print_nodes(nodes: list[Node]):
    for node in nodes:
        if node.datapoint is not None:
            print(node.datapoint.coordinate)
        else:
            print('Left Rectangle: ', node.left_rectangle)
            print_nodes([node.left_child])
            print('Right Rectangle: ', node.right_rectangle)
            print_nodes([node.right_child])

In [59]:
def check_longest_tao(k: int, starting_value: float, starting_timestamp: float, start_time_index: float, end_time_index: float, timestamps: np.ndarray, tree: KDTree ) -> float:
    mid = (start_time_index + end_time_index) // 2
    if mid == start_time_index or mid == end_time_index:
        c_nodes1 = tree.query_canonical( [[starting_timestamp, starting_value], [timestamps[start_time_index], 9999]] )
        p_above1 = 0
        for node in c_nodes1:
            p_above1 += node.children if node.children > 0 else 1
        tau0 = timestamps[start_time_index] - starting_timestamp - 0.01 if timestamps[start_time_index] - starting_timestamp > 0 else 0
        tau1 = timestamps[end_time_index] - starting_timestamp - 0.01 if timestamps[end_time_index] - starting_timestamp > 0 else 0
        return tau0 if p_above1 == k else tau1
    
    curr_timestamp = timestamps[mid]
    canonical_nodes = tree.query_canonical( [[starting_timestamp, starting_value], [curr_timestamp, 9999]] )
    if len(canonical_nodes) == 0:
        return 0
    points_above = 0
    for node in canonical_nodes:
        points_above += node.children if node.children > 0 else 1
    if points_above == k:
        return check_longest_tao(k, starting_value, starting_timestamp, start_time_index, mid, timestamps, tree)
    elif points_above > k:
        return check_longest_tao(k, starting_value, starting_timestamp, start_time_index, mid, timestamps, tree)
    elif points_above < k:
        return check_longest_tao(k, starting_value, starting_timestamp, mid, end_time_index, timestamps, tree)

def unknown_tao_tree_build(k: int, window: list, data: pd.DataFrame, tree: KDTree) -> KDTree or None:
    data = data.query(f"timestamp > {window[0]} and timestamp < {window[1]}")
    unique_timestamps = data.timestamp.unique()
    unique_timestamps = unique_timestamps
    unique_timestamps = sorted(unique_timestamps)
    for row in data.itertuples():
        timestamp_index = unique_timestamps.index(row.timestamp)
        timestamps = unique_timestamps[timestamp_index:]
        x = check_longest_tao(k, row.fg+0.01, row.timestamp, 0, len(timestamps)-1, timestamps, tree)
        if x != 0:
            print('data: ', row.timestamp, row.fg, row.pid, row.timestamp+x, x)

In [60]:
unknown_tao_tree_build(k, window, data, tree1)

data:  19361211 4 30 19370111.99 8900.99
data:  19361211 1 77 19361211.99 0.99
data:  19361211 1 178 19361211.99 0.99
data:  19361211 0 185 19361211.99 0.99
data:  19361211 1 263 19361211.99 0.99
data:  19361211 1 290 19361211.99 0.99
data:  19361211 3 304 19361216.99 5.99
data:  19361211 2 340 19361215.99 4.99
data:  19361211 0 343 19361211.99 0.99
data:  19361211 0 421 19361211.99 0.99
data:  19361212 3 30 19361216.99 4.99
data:  19361212 1 77 19361215.99 3.99
data:  19361212 2 178 19361216.99 4.99
data:  19361212 0 185 19361215.99 3.99
data:  19361212 1 263 19361215.99 3.99
data:  19361212 4 290 19370111.99 8899.99
data:  19361212 0 304 19361215.99 3.99
data:  19361212 2 340 19361216.99 4.99
data:  19361212 0 343 19361215.99 3.99
data:  19361212 1 421 19361215.99 3.99
data:  19361217 5 304 19371212.99 9995.99
data:  19361217 2 340 19370105.99 8888.99
data:  19370106 3 30 19370106.99 0.99
data:  19370106 1 77 19370106.99 0.99
data:  19370106 4 125 19370207.99 101.99
data:  19370106 1

KeyboardInterrupt: 